# LSTM Grid Search -
นำ LSTM มา Grid Search โดยใช้วิธี Rolling Window CV แต่ละ Fold (รอบการประเมิน) จะถูกเทรนด้วยวิธี multi step direct forecasting 

## ตั้งค่าและนำเข้า Libraries

In [1]:
# 1. Setup & Import Libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import random
import itertools
import time
import copy
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

# Device Configuaration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using Device: {DEVICE}")

# Settings
SEED = 42
N_JOBS = 64
VALID_SIZE = 155

N_SPLITS = 5
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(SEED)

Using Device: cuda


## โหลดข้อมูล

In [2]:
# 2. Data Loading & Preprocessing
sheet_id = "1-hzX_qRFjS7TIhWkTsrWPx7M_cFATCQxvWlRmo97Wac"
sheet_gid = "0"
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={sheet_gid}"

print("Loading data")
data = pd.read_csv(csv_url)
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data = data.sort_values('Date').reset_index(drop=True)

# Parameters
n = len(data)
n_train_full = 618 
y = data['Y'].values
dy = np.diff(y)
dy_train_full = dy[:n_train_full - 1]

print(f"Data Loaded: n={n}, n_train_full={n_train_full}")
print(f"Diff-1 Series Length (for CV): {len(dy_train_full)}")

Loading data
Data Loaded: n=773, n_train_full=618
Diff-1 Series Length (for CV): 617


/tmp/ipykernel_5205/509359041.py:8: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)


## ออกแบบการ Cross Validation ตามแนวทาง Rolling Window CV

In [3]:
# 3. CV process

def create_sequences_direct(series, window, horizon, end_index):
    """
    Creates X, y for Direct strategy at specific horizon h.
    """
    max_start = end_index - window - horizon + 1
    if max_start < 0:
        # Not enough data for this h
        return np.array([]), np.array([])
        
    X = []
    y = []
    
    for i in range(max_start + 1):
        seq_x = series[i : i + window]
        target = series[i + window + horizon - 1]
        X.append(seq_x)
        y.append(target)
        
    return np.array(X), np.array(y)

def make_cv_splits(n_seq, valid_size=155, n_splits=5):
    """
    Rolling Overlap CV splits 
    """
    
    usable = n_seq
    if usable <= valid_size + 1:
        raise ValueError("usable seq <= valid_size")
        
    train_size = int(np.floor((usable - valid_size) * 3/4))
    
    start_min = 0
    start_max = usable - (train_size + valid_size)
    
    if start_max < start_min:
        raise ValueError("start_max < start_min")
        
    if n_splits == 1:
        step_s = 0
    else:
        step_s = max(1, int(np.floor((start_max - start_min) / (n_splits - 1))))
        
    splits = []
    for k in range(n_splits):
        s = start_min + k * step_s
        tr_start = s
        tr_end = s + train_size
        va_start = tr_end
        va_end = tr_end + valid_size
        
        splits.append((np.arange(tr_start, tr_end), np.arange(va_start, va_end)))
        
    return splits

## ออกแบบโครงข่ายประสาทเทียมแบบ LSTM

In [4]:
# 4. Model Definition

class LSTMDirect(nn.Module):
    def __init__(self, lstm1_units, lstm2_units, dense1_units, dense2_units, window_size):
        super(LSTMDirect, self).__init__()
        self.lstm1 = nn.LSTM(input_size=1, hidden_size=lstm1_units, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=lstm1_units, hidden_size=lstm2_units, batch_first=True)
        self.fc1 = nn.Linear(lstm2_units, dense1_units)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(dense1_units, dense2_units)
        self.relu2 = nn.ReLU()
        self.fc_out = nn.Linear(dense2_units, 1)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        out = out[:, -1, :]
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc_out(out)
        return out

## Function ดำเนินการ Grid Search

In [5]:
# 5. Grid Search
# 1 Combination * 5 Folds * 155 Horizons = 775 Models trained.

def train_model_for_horizon(train_series, h_target, w, params, end_idx_train, device):
    
    X, y = create_sequences_direct(train_series, w, h_target, end_index=len(train_series)-1)
    
    if len(X) == 0: return None
    
    X_t = torch.tensor(X.reshape(-1, w, 1), dtype=torch.float32).to(device)
    y_t = torch.tensor(y.reshape(-1, 1), dtype=torch.float32).to(device)
    
    dataset = TensorDataset(X_t, y_t)
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    
    model = LSTMDirect(
        int(params['lstm1_units']), int(params['lstm2_units']),
        int(params['dense1_units']), int(params['dense2_units']),
        w
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    criterion = nn.MSELoss()
    
    epochs = int(params['epochs'])
    best_loss = float('inf')
    best_state = None
    patience = 10
    counter = 0
    
    model.train()
    for ep in range(epochs):
        for xb, yb in loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_state = copy.deepcopy(model.state_dict())
            counter = 0
        else:
            counter += 1
            if counter >= patience: break
            
    if best_state:
        model.load_state_dict(best_state)
        
    return model

def evaluate_combination_true_direct(params, full_dy, device_name):
    """
    Evaluates params using Direct Strategy.
    For each fold: Train 155 models (one per horizon).
    """
    device = torch.device(device_name)
    w = int(params['window_size'])
    
    # Make Splits on the full available history
    splits = make_cv_splits(len(full_dy), VALID_SIZE, N_SPLITS)
    
    fold_rmses = []
    
    for fold_idx, (tr_idx, va_idx) in enumerate(splits):
        # Data for this fold
        fold_train_dy = full_dy[tr_idx]
        fold_valid_dy = full_dy[va_idx]
        
        # Input for prediction is the Last window of the training set
        last_window = fold_train_dy[-w:]
        last_window_t = torch.tensor(last_window.reshape(1, w, 1), dtype=torch.float32).to(device)
        
        # List to store predictions for h=1 ถึง 155
        fold_preds = []
        
        # Train 155 models for this fold
        for h in range(1, VALID_SIZE + 1):
            model_h = train_model_for_horizon(fold_train_dy, h, w, params, len(fold_train_dy)-1, device)
            
            if model_h is None:
                fold_preds.append(0.0)
                continue
                
            model_h.eval()
            with torch.no_grad():
                pred = model_h(last_window_t).item()
                fold_preds.append(pred)
                        
        rmse = np.sqrt(np.mean((fold_valid_dy - np.array(fold_preds))**2))
        fold_rmses.append(rmse)
        print(f"   Fold {fold_idx+1}/{N_SPLITS} RMSE: {rmse:.4f}")
        
    return np.mean(fold_rmses)

# เริ่มดำเนินการและผลลัพธ์การ Grid Search

In [6]:
# 6. Execution

hyper_grid = {
    'lstm1_units': [50,100],
    'lstm2_units': [32,64],
    'dense1_units': [32,64],
    'dense2_units': [32,64],
    'learning_rate': [0.001,0.01],
    'epochs': [50],
    'window_size': [10,20]
}

keys, values = zip(*hyper_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
print(f"Total Combinations: {len(combinations)}")
print(f"Models per combo: {N_JOBS * VALID_SIZE} (approx)")

# Parallel Execution
N_GPUS = torch.cuda.device_count() if torch.cuda.is_available() else 0

def safe_evaluate(params, worker_id):
    if N_GPUS > 0:
        gpu_id = worker_id % N_GPUS
        device_str = f'cuda:{gpu_id}'
    else:
        device_str = 'cpu'
    return evaluate_combination_true_direct(params, dy_train_full, device_str)

results = Parallel(n_jobs=N_JOBS)(
    delayed(safe_evaluate)(params, i) 
    for i, params in enumerate(combinations)
)

# Process Results
best_score = float('inf')
best_params = None

for i, score in enumerate(results):
    params = combinations[i]
    print(f"Params: {params}, CV-RMSE: {score:.4f}")
    if score < best_score:
        best_score = score
        best_params = params

print("\nGrid Search Complete.")
print("Best Params:", best_params)
print("Best RMSE:", best_score)

Total Combinations: 64
Models per combo: 9920 (approx)
   Fold 1/5 RMSE: 15.9542
   Fold 1/5 RMSE: 16.6427
   Fold 1/5 RMSE: 16.2784
   Fold 1/5 RMSE: 16.3739
   Fold 1/5 RMSE: 16.2115
   Fold 1/5 RMSE: 16.5612
   Fold 1/5 RMSE: 17.0200
   Fold 1/5 RMSE: 15.8277
   Fold 1/5 RMSE: 17.4752
   Fold 1/5 RMSE: 16.3356
   Fold 1/5 RMSE: 16.3870
   Fold 1/5 RMSE: 16.1251
   Fold 1/5 RMSE: 16.6337
   Fold 1/5 RMSE: 16.3716
   Fold 1/5 RMSE: 16.1016
   Fold 1/5 RMSE: 15.1742
   Fold 1/5 RMSE: 18.0038
   Fold 1/5 RMSE: 17.2725
   Fold 1/5 RMSE: 18.1351
   Fold 1/5 RMSE: 17.8041
   Fold 1/5 RMSE: 17.7704
   Fold 1/5 RMSE: 17.5313
   Fold 1/5 RMSE: 20.3833
   Fold 1/5 RMSE: 17.6474
   Fold 1/5 RMSE: 19.3362
   Fold 1/5 RMSE: 18.1103
   Fold 1/5 RMSE: 18.4423
   Fold 1/5 RMSE: 18.2945
   Fold 1/5 RMSE: 18.5608
   Fold 1/5 RMSE: 17.6143
   Fold 1/5 RMSE: 19.9640
   Fold 1/5 RMSE: 17.6357
   Fold 1/5 RMSE: 18.4967
   Fold 1/5 RMSE: 17.6160
   Fold 1/5 RMSE: 18.8856
   Fold 1/5 RMSE: 17.6426
   Fold 1